In [217]:
# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
mapints = aoc_utils.mapints
nums = mapints
cat = aoc_utils.cat


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [218]:
# Useful imports
import re
from collections import defaultdict, deque
import heapq
import functools
import queue
import itertools
import math
from functools import cache
import time
import operator

In [ ]:
data = get_input(1, 2022)

xs = [sum(map(int, grp)) for grp in list(aoc_utils.split_list(data))]
p1 = max(xs)
p2 = sum(list(reversed(sorted(xs)))[:3])
assert p1,p2 == (66616, 199172)

In [ ]:
data = get_input(2, 2022)
data

TOWIN = {
  'A': 'Y',
  'B': 'Z',
  'C': 'X',
}
TOLOSE = {
  'A': 'Z',
  'B': 'X',
  'C': 'Y',
}

ELF = 'ABC'
YOU = 'XYZ'

p1 = 0
for line in data:
  lhs,rhs = line.split(' ')
  p1 += 1 + YOU.find(rhs)
  if ELF.index(lhs) == YOU.index(rhs): # draw
    p1 += 3
  else:
    if TOWIN[lhs] == rhs:
      p1 += 6
    else:
      p1 += 0

p2 = 0
for line in data:
  l,r = line.split(" ")
  if r == 'X': #lose
    shape = TOLOSE[l]
    p2 += 0
  elif r == 'Y': #draw
    shape = YOU[ELF.find(l)]
    p2 += 3
  elif r == 'Z': # win
    shape = TOWIN[l]
    p2 += 6
  p2 += 1 + YOU.find(shape)

assert p1,p2 == (13682, 12881)

In [ ]:
data = get_input(3, 2022)
data

from string import ascii_letters

p1 = 0
for line in data:
  l,r = line[:len(line)//2],line[len(line)//2:]
  xs = set(l) & set(r)
  assert len(xs) == 1
  p1 += 1 + ascii_letters.index(list(xs)[0])
p1

p2 = 0
for grp in aoc_utils.chunker(data, 3):
  xs = set(grp[0]) & set(grp[1]) & set(grp[2])
  assert len(xs) == 1
  p2 += 1 + ascii_letters.index(list(xs)[0])
assert (p1, p2) ==  (8233, 2821)

In [ ]:
data = get_input(4,2022)
data

p1 = p2 = 0
for line in data:
  a,b = [list(map(int, x.split('-'))) for x in line.split(',')]
  p1 += any(x[0] >= y[0] and x[1] <= y[1] for x,y in [(a,b),(b,a)])
  ra,rb = range(a[0],a[1]+1),range(b[0],b[1]+1)
  p2 += any(x[0] in r and x[1] in r for x,r in [(a,rb),(b,ra)])

assert p1,p2 == (515,883)

In [ ]:
data = get_input(5,2022)

towerdata,moves = aoc_utils.split_list(data)

def mk_towers(towerdata):
  toweridxs = mapints(towerdata[-1])
  towers = defaultdict(list)
  for line in towerdata[:-1]: # last line is the indices
    for idx in toweridxs:
      pos = (idx-1)*4 + 1
      if pos < len(line) and line[pos] != ' ':
        towers[idx].append(line[pos])
  return {idx: list(reversed(tower)) for idx,tower in towers.items()}

towers1 = mk_towers(towerdata)
towers2 = mk_towers(towerdata)

for line in moves:
  cnt,start,end = mapints(line)

  assert len(towers1[start]) >= cnt
  assert len(towers2[start]) >= cnt

  # p1
  for _ in range(cnt): towers1[end].append(towers1[start].pop())

  # p2
  towers2[end] += towers2[start][-cnt:]
  towers2[start] = towers2[start][:-cnt]

p1 = cat([towers1[idx][-1] for idx in sorted(towers1.keys())])
p2 = cat([towers2[idx][-1] for idx in sorted(towers2.keys())])

assert (p1,p2) == ('LJSVLTWQM', 'BRQWDBBJM')

In [ ]:
data = get_input(6, 2022)[0]

p1 = None
p2 = None
for idx,ch in enumerate(data):
  if idx > 3 and p1 is None:
    if len(set(data[idx-4:idx])) == 4:
      p1 = idx
  if idx > 13 and p2 is None:
    if len(set(data[idx-14:idx])) == 14:
      p2 = idx

assert (p1,p2) == (1343, 2193)

In [6]:
from pathlib import PurePath
data = get_input(7, 2022)

dirs = defaultdict(set)
cwd = None
mode = None
for line in data:
  if line.startswith('$ cd'):
    mode = None
    target = line.split(' ')[-1]
    if target == '/':
      cwd = PurePath('/')
    elif target == '..':
      cwd = cwd.parent
    else:
      cwd = cwd / target
  elif line.startswith('$ ls'):
    mode = 'ls'
  else:
    assert mode == 'ls'
    if line.startswith('dir'): pass
    else:
      size = mapints(line)[0]
      file = line.split(' ')[-1]
      dirs[cwd].add( (file, size) )


@cache
def dirsize(target):
  return (
    sum(f[1] for f in dirs[target]) +
    sum(dirsize(child) for child in dirs if child.parent == target and child != target)
  )

all_sizes = {dir:dirsize(dir) for dir in dirs}
p1 = sum(s for s in all_sizes.values() if s <= 100_000)

TOTAL = 70000000
NEEDED = 30000000
USED = TOTAL - dirsize(PurePath('/'))

p2 = min(s for s in all_sizes.values() if s >= NEEDED-USED)

assert p1,p2 == (1783610, 4370655)

In [10]:
timer = aoc_utils.start_timer()

In [54]:
data = get_input(8, 2022)
testdata = """30373
25512
65332
33549
35390""".split('\n')

colmax = len(data[0])
rowmax = len(data)
visible = set()

def get_row(idx):
  return list(map(int, data[idx]))

def get_col(idx):
  return list(map(int, [data[rowidx][idx] for rowidx in range(rowmax)]))

def filter_visible(arr):
  maxes = list(itertools.accumulate(arr, max))
  return [cur>prev for (cur,prev) in zip( maxes, [-1] + maxes[:-1])]

for ridx in range(rowmax):
  row = get_row(ridx)
  for cidx,is_vis in enumerate(filter_visible(row)):
    if is_vis: visible.add((cidx,ridx))
  for revcidx,is_vis in enumerate(filter_visible(row[::-1])):
    if is_vis: visible.add((colmax-1-revcidx,ridx))

for cidx in range(colmax):
  col = get_col(cidx)
  for ridx,is_vis in enumerate(filter_visible(col)):
    if is_vis: visible.add((cidx,ridx))
  for revridx,is_vis in enumerate(filter_visible(col[::-1])):
    if is_vis: visible.add((cidx, rowmax-1-revridx))

p1 = len(visible)

def view_dist(arr, val):
  idx = aoc_utils.findindex(arr, lambda v: v >= val)
  return len(arr) if idx is None else (idx + 1)

p2 = 0
for (cidx,ridx) in itertools.product(range(colmax),range(rowmax)):
  row,col = get_row(ridx),get_col(cidx)
  l,r,u,d = row[:cidx][::-1],row[cidx+1:],col[:ridx][::-1],col[ridx+1:]
  curval = int(data[ridx][cidx])
  scenic = functools.reduce(operator.mul, [view_dist(arr,curval) for arr in [u,r,l,d]])
  p2 = max(p2, scenic)

assert (p1,p2) == (1854, 527340)


527340

In [5]:
data = get_input(9,2022)
testdata = """R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2""".split('\n')

CARDINAL_DELTAS = aoc_utils.CARDINAL_DELTAS
manhattan_distance = aoc_utils.manhattan_distance

def mv_tail(h,t):
  tx,ty = t
  hx,hy = h
  md = manhattan_distance(t,h)
  if md in [0,1]: return t
  if md == 2 and tx != hx and ty != hy: return t

  dx = 1 if tx < hx else -1
  dy = 1 if ty < hy else -1

  if tx == hx:
    return (tx, ty+dy)
  if ty == hy:
    return (tx+dx,ty)
  return (tx+dx,ty+dy)

def mv_head(h,dir):
  x,y = h
  dx,dy = CARDINAL_DELTAS[dir]
  return (x+dx,y+dy)

def mv_rope(rp,dir):
  return list(itertools.accumulate(rp[1:], func=mv_tail, initial=mv_head(rp[0], dir)))

ROPE = [(0,0)] * 10
p1seen = set([(0,0)])
p2seen = set([(0,0)])
for line in data:
  cnt = mapints(line)[0]
  dir = line[0]
  for _ in range(cnt):
    ROPE = mv_rope(ROPE, dir)
    p1seen.add(ROPE[1])
    p2seen.add(ROPE[-1])
p1 = len(p1seen)
p2 = len(p2seen)

print(p1,p2)
assert (p1,p2) == (6081, 2487)


6081 2487


In [139]:
data = get_input(10,2022)

def run(data):
  X = 1
  yield X
  for line in data:
    yield X
    if 'addx' in line:
      yield X
      X += int(line.split(' ')[1])
  yield X

xs = list(run(data))
p1 = sum(idx*v for idx,v in enumerate(xs) if idx % 40 == 20)

G = {}
rows = 6
rowlen = 40
for y in range(rows):
  for x in range(rowlen):
    cycle_idx = 1 + x + rowlen*y
    pos = (x,y)
    spritemid = xs[cycle_idx]
    pixel = '⬛' if abs(x-spritemid) > 1 else '⬜'
    G[pos] = pixel


print(f"p1 {p1}")
assert p1 == 15880

print("p2:")
for y in range(rows):
  for x in range(rowlen):
    print(G[(x,y)], end='')
  print('')

p1 15880
p2:
⬜⬜⬜⬛⬛⬜⬛⬛⬛⬛⬛⬜⬜⬛⬛⬜⬜⬜⬜⬛⬜⬛⬛⬜⬛⬛⬜⬜⬛⬛⬜⬜⬜⬜⬛⬛⬜⬜⬛⬛
⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛⬜⬛⬜⬛⬛⬜⬛⬛⬜⬛⬛⬛⬛⬜⬛⬜⬛⬛⬜⬛
⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬜⬜⬛⬛⬜⬜⬛⬛⬛⬜⬛⬛⬜⬛⬛⬛⬜⬛⬛⬜⬛⬛⬛⬛
⬜⬜⬜⬛⬛⬜⬛⬛⬛⬛⬜⬛⬜⬜⬛⬜⬛⬛⬛⬛⬜⬛⬜⬛⬛⬜⬜⬜⬜⬛⬛⬜⬛⬛⬛⬜⬛⬜⬜⬛
⬜⬛⬛⬛⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛⬜⬛⬜⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛
⬜⬛⬛⬛⬛⬜⬜⬜⬜⬛⬛⬜⬜⬜⬛⬜⬛⬛⬛⬛⬜⬛⬛⬜⬛⬜⬛⬛⬜⬛⬜⬜⬜⬜⬛⬛⬜⬜⬜⬛


In [147]:
timer = aoc_utils.start_timer()

In [299]:
data = get_input(11,2022)

testdata = """Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1""".split('\n')

OPS = { '+': operator.add, '*': operator.mul }
def parse(data):
  monkeys = {}
  for idx,grp in enumerate(aoc_utils.split_list(data)):
    inspect = 0
    items = nums(grp[1])
    op,v = grp[2].split('Operation: new = old ')[1].split(' ')
    test,true,false = nums(' '.join(grp[3:]))
    monkeys[idx] = [inspect,op,v,test,true,false,items]
  return monkeys


monkeys = parse(data)
PROD = math.prod(map(lambda m: m[3], monkeys.values()))

def rnd(monkeys,is_p2=False):
  for idx in range(len(monkeys.keys())):
    (inspect,op,v,test,true,false,items) = monkeys[idx]
    monkeys[idx][0] += len(items) # update inspect
    monkeys[idx][-1] = [] # clear items
    for lvl in items:
      lvl = OPS[op](lvl, int(v) if v.isdigit() else lvl)
      if not is_p2:
        lvl //= 3
      lvl = lvl % PROD
      target = true if lvl % test == 0 else false
      monkeys[target][-1].append(lvl)
  return monkeys

monkeys = parse(data)
for _ in range(20):
  monkeys = rnd(monkeys)
a,b = list(sorted(map(lambda m:m[0], monkeys.values()),reverse=True))[:2]
p1 = a * b

monkeys = parse(data)
for _ in range(10_000):
  monkeys = rnd(monkeys,is_p2=True)
a,b = list(sorted(map(lambda m:m[0], monkeys.values()),reverse=True))[:2]
p2 = a * b

# p1 in 2123s
print("p1",p1)
assert p1 == 62491

# p2 in 2778s
print("p2",p2)
assert p2 == 17408399184

p1 62491
p2 17408399184


In [302]:
timer = aoc_utils.start_timer()

In [316]:
data = get_input(12,2022)

S = None
E = None
G = {}
from string import ascii_letters as alpha
for col,line in enumerate(data):
  for row,ch in enumerate(line):
    pos = (row,col)
    if ch == 'S':
      S = pos
      ch ='a'
    if ch == 'E':
      E = pos
      ch = 'z'
    G[(row,col)] = alpha.index(ch)

def solve(S, E, G):
  from heapq import heappop,heappush
  search = []
  moves = 0
  seen = set()
  heappush(search, (moves, S))
  while len(search):
    (moves, pos) = heappop(search)
    if pos == E:
      return moves
    for n in aoc_utils.neighbors4(pos):
      if n in G and G[n] <= (G[pos]+1) and n not in seen:
        seen.add(n)
        heappush(search, (moves+1,n))

p1 = solve(S, E, G)

p2 = min(filter(lambda x: x is not None,
  [solve(pos, E, G) for (pos, v) in G.items() if v == alpha.index('a')]
))

print(f"p1 {p1}") # 624s
print(f"p2 {p2}") # 859s

assert p1 == 420
assert p2 == 414




p1 420
p2 414
